<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import re

warnings.filterwarnings('ignore')

os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('validation.csv')

  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 36.5MB/s]
  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 71.4MB/s]


In [51]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [52]:
train_df['label']  = train_df['Labels'].apply(lambda x :  labelsplit(x))

In [53]:
onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in train_df.label[i] : # j = value   
        onehot[i,j]  = 1

onehot = pd.DataFrame(onehot, columns=['0','1','2','3','4','5','6','7','8','9','10'])
onehot[:3]

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
answer = onehot[:2500]

In [57]:
train_df = pd.concat(
    [train_df, onehot], axis=1
).drop(['Labels', 'label'],axis=1)

train_df.head()

,ID,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,Im waiting for someone to say to me that all t...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,He is a liar. Proven day night. Time again. Li...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [58]:
test_df.head()

,ID,Tweet
0,5001,Forgot to a math test and I was failing but my...
1,5002,Corona effected came from Delhi in Bihar lakhi...
2,5003,Make CORONA END NA
3,5004,Imagine if the coronavirus pandemic was a big ...
4,5005,Howdy Ana. Where can I get married during the ...


In [59]:
cols_target = ['0','1','2','3','4','5','6','7','8','9','10']

In [60]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [61]:
train_df['Tweet'] = train_df['Tweet'].map(lambda x : clean_text(x))
test_df['Tweet'] = test_df['Tweet'].map(lambda x : clean_text(x))

In [62]:
x = train_df.Tweet
test_x = test_df.Tweet
print(x.shape, test_x.shape)

(5000,) (2500,)


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer(max_features=2000, stop_words = 'english')
vector

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [64]:
x_dtm = vector.fit_transform(x)
x_dtm

<5000x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 36699 stored elements in Compressed Sparse Row format>

In [65]:
test_x_dtm = vector.fit_transform(test_x)
test_x_dtm

<2500x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 18721 stored elements in Compressed Sparse Row format>

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=23.0)

def add_feature(x, feature_to_add) :
    from scipy.sparse import csr_matrix, hstack
    return hstack([x, csr_matrix(feature_to_add).T], 'csr')

In [67]:
for label in cols_target :
    print('진행중인 label : {}'.format(label))
    y = train_df[label]

    logreg.fit(x_dtm, y)

    y_pred_x = logreg.predict(x_dtm)
    print('training accuracy : {}'.format(accuracy_score(y, y_pred_x)))

    test_y = logreg.predict(test_x_dtm)
    test_y_prob = logreg.predict_proba(test_x_dtm)[:,1]
    answer[label] = test_y_prob

    x_dtm = add_feature(x_dtm, y)
    test_x_dtm = add_feature(test_x_dtm, test_y)


진행중인 label : 0
training accuracy : 0.91
진행중인 label : 1
training accuracy : 0.9908
진행중인 label : 2
training accuracy : 0.9932
진행중인 label : 3
training accuracy : 0.9418
진행중인 label : 4
training accuracy : 0.9276
진행중인 label : 5
training accuracy : 0.9156
진행중인 label : 6
training accuracy : 0.899
진행중인 label : 7
training accuracy : 0.9872
진행중인 label : 8
training accuracy : 0.9914
진행중인 label : 9
training accuracy : 0.9186
진행중인 label : 10
training accuracy : 0.909


In [68]:
answer

,0,1,2,3,4,5,6,7,8,9,10
0,0.706967,0.050172,0.016931,0.436160,0.000973,0.000544,0.968109,0.003876,0.001730,0.026239,0.128515
1,0.345392,0.002428,0.007835,0.001920,0.004563,0.615509,0.864514,0.000668,0.057362,0.006682,0.070434
2,0.399551,0.002514,0.001632,0.020699,0.435716,0.303436,0.522513,0.001201,0.008436,0.167137,0.475280
3,0.007789,0.001761,0.016528,0.024252,0.578793,0.107346,0.375314,0.028228,0.015406,0.006154,0.764422
4,0.365481,0.182304,0.001165,0.058717,0.005517,0.173234,0.950153,0.046860,0.131368,0.795109,0.504939
...,...,...,...,...,...,...,...,...,...,...,...
2495,0.318635,0.037106,0.001446,0.044223,0.183765,0.124077,0.583980,0.021988,0.313698,0.131999,0.861373
2496,0.105506,0.002581,0.004080,0.051937,0.000450,0.762946,0.787648,0.126810,0.019689,0.099284,0.002715
2497,0.071441,0.004765,0.001227,0.136430,0.013968,0.493069,0.930962,0.011968,0.011925,0.031211,0.005633
2498,0.900600,0.213629,0.011244,0.593114,0.600283,0.005613,0.000223,0.003663,0.051207,0.047335,0.018126


In [69]:
test_df.Tweet[0]

'forgot to a math test and i was failing but my teacher opening it back up maybe corona aint so bad'

In [70]:
answer[:10]

,0,1,2,3,4,5,6,7,8,9,10
0,0.706967,0.050172,0.016931,0.436160,0.000973,0.000544,0.968109,0.003876,0.001730,0.026239,0.128515
1,0.345392,0.002428,0.007835,0.001920,0.004563,0.615509,0.864514,0.000668,0.057362,0.006682,0.070434
2,0.399551,0.002514,0.001632,0.020699,0.435716,0.303436,0.522513,0.001201,0.008436,0.167137,0.475280
3,0.007789,0.001761,0.016528,0.024252,0.578793,0.107346,0.375314,0.028228,0.015406,0.006154,0.764422
4,0.365481,0.182304,0.001165,0.058717,0.005517,0.173234,0.950153,0.046860,0.131368,0.795109,0.504939
5,0.059250,0.079649,0.005008,0.010047,0.033111,0.008363,0.848552,0.000146,0.056431,0.465574,0.206700
6,0.057218,0.005007,0.003936,0.010148,0.021814,0.116346,0.914386,0.040920,0.004765,0.112958,0.618392
7,0.254092,0.121780,0.005701,0.002689,0.045785,0.264998,0.349285,0.022389,0.410976,0.009755,0.743049
8,0.044185,0.002576,0.048756,0.503181,0.083776,0.119195,0.241750,0.110905,0.015059,0.531451,0.953439
9,0.211754,0.001424,0.000664,0.966017,0.167808,0.370238,0.960435,0.020558,0.000967,0.083992,0.440795


In [71]:
answer[10:20]

,0,1,2,3,4,5,6,7,8,9,10
10,0.363029,0.001640,0.001516,0.000832,0.022986,0.005777,0.732652,0.340196,0.016204,0.794459,0.171866
11,0.460505,0.014531,0.001484,0.002600,0.143625,0.083275,0.370126,0.333070,0.381262,0.039326,0.903851
12,0.702669,0.300963,0.004923,0.102454,0.014518,0.005126,0.384541,0.018484,0.531673,0.100554,0.007179
13,0.012693,0.000244,0.005209,0.376008,0.197238,0.096877,0.647755,0.043051,0.005410,0.553519,0.008918
14,0.030472,0.000447,0.002559,0.004194,0.256584,0.319013,0.556540,0.010400,0.092654,0.842335,0.068728
15,0.001139,0.003156,0.003807,0.000748,0.007999,0.015702,0.756745,0.013038,0.003999,0.002944,0.929252
16,0.036747,0.007340,0.002324,0.064656,0.049393,0.115310,0.665095,0.017913,0.094871,0.010824,0.598952
17,0.061938,0.002016,0.091211,0.065769,0.164493,0.003518,0.918759,0.003468,0.013713,0.447260,0.287793
18,0.429707,0.002563,0.001686,0.002244,0.009855,0.172182,0.021370,0.005862,0.200343,0.836751,0.982578
19,0.016677,0.001830,0.000864,0.007836,0.237089,0.035956,0.025043,0.009980,0.089361,0.864984,0.309226


In [72]:
answer

,0,1,2,3,4,5,6,7,8,9,10
0,0.706967,0.050172,0.016931,0.436160,0.000973,0.000544,0.968109,0.003876,0.001730,0.026239,0.128515
1,0.345392,0.002428,0.007835,0.001920,0.004563,0.615509,0.864514,0.000668,0.057362,0.006682,0.070434
2,0.399551,0.002514,0.001632,0.020699,0.435716,0.303436,0.522513,0.001201,0.008436,0.167137,0.475280
3,0.007789,0.001761,0.016528,0.024252,0.578793,0.107346,0.375314,0.028228,0.015406,0.006154,0.764422
4,0.365481,0.182304,0.001165,0.058717,0.005517,0.173234,0.950153,0.046860,0.131368,0.795109,0.504939
...,...,...,...,...,...,...,...,...,...,...,...
2495,0.318635,0.037106,0.001446,0.044223,0.183765,0.124077,0.583980,0.021988,0.313698,0.131999,0.861373
2496,0.105506,0.002581,0.004080,0.051937,0.000450,0.762946,0.787648,0.126810,0.019689,0.099284,0.002715
2497,0.071441,0.004765,0.001227,0.136430,0.013968,0.493069,0.930962,0.011968,0.011925,0.031211,0.005633
2498,0.900600,0.213629,0.011244,0.593114,0.600283,0.005613,0.000223,0.003663,0.051207,0.047335,0.018126


In [78]:
answer_2 = answer

In [81]:
answer_2[answer_2 >= 0.5] = 1
answer_2[answer_2 < 0.5] = 0
answer_2.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [121]:
t = np.ones(2500,)
example = pd.DataFrame(t, columns=['Example']).astype(int)
example.Example[0] = '0 6'
example.Example[1] = '5 6'
example.Example[2] = '6'
example.Example[3] = '4 10'
example.Example[4] = '6 9 10'
example[:5]

,Example
0,0 6
1,5 6
2,6
3,4 10
4,6 9 10
